# Importing training Data
TODO: dealing with test data loading

In [1]:
import numpy as np
from glob import glob
import omegaconf
from utils.data import DataLoader
import matplotlib.pyplot as plt
import pandas as pd


In [2]:
from torch.utils.tensorboard import SummaryWriter


/home/bdudas/anaconda3/envs/pct/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
writer = SummaryWriter()

In [3]:
conf = omegaconf.OmegaConf.load('configs/baseConfig.yaml')

In [4]:
train_loader = DataLoader('data/train',**conf.LoaderParams)

In [5]:
batch = next(iter(train_loader))

In [4]:
train_loader.allPaths[:5]

array(['data/train/wpt_130/1913.npy', 'data/train/wpt_170/3286.npy',
       'data/train/wpt_160/4194.npy', 'data/train/wpt_90/5883.npy',
       'data/train/wpt_160/3218.npy'], dtype='<U27')

# Model testings

In [5]:
from utils.model import PosPredictor

/home/bdudas/anaconda3/envs/pct/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [125]:
def SinkhornMatching(distMx,temp=0.59,n_iter=10):
    S = torch.softmax(-distMx/temp,dim=1)
    for i in range(n_iter):
        S /= torch.sum(S,dim=1,keepdim=True)
        S /= torch.sum(S,dim=2,keepdim=True)
    return S
def CorrectionMask(S):
    xpreds = torch.argmax(S,dim=1)
    ypreds = torch.argmax(S,dim=2)
    return xpreds == ypreds


In [148]:
conf.LoaderParams.norm = False
conf.LoaderParams.ParticleNumber = 200
train_loader = DataLoader('data/train',**conf.LoaderParams)

In [179]:
batch = next(iter(train_loader))
bidx = 64
tl = 1
batch.shape

(256, 200, 41, 3)

In [188]:
x = torch.from_numpy(batch[:bidx,:,tl])
y = torch.from_numpy(batch[:bidx,:,tl+1])
y.shape

torch.Size([64, 200, 3])

In [189]:
#x [x == 0] = 1000
#y[y == 0] = 1000

In [190]:
y.nelement()

38400

In [218]:
distMx = torch.cdist(x,y)
S = SinkhornMatching(distMx,temp = 0.5)
LikelyMatches = torch.argmax(S,dim=1)
mask = CorrectionMask(S)

In [219]:
torch.sum(mask == False)

tensor(3465)

In [220]:
#LikelyMatches = torch.argmax(S,dim=1)
torch.sum(x[0,LikelyMatches[0]] == x[0])/x[0].nelement()

tensor(0.7550)

In [221]:

x[0].shape,mask[0].shape,torch.sum(mask[0])

(torch.Size([200, 3]), torch.Size([200]), tensor(144))

In [222]:
acc = 0
numRec = 0
for b in range(y.shape[0]):
    numReconstructed = torch.sum(mask[b])
    numRec += numReconstructed
    truepoints = y[b][mask[b]]
    comparepoints = y[b,LikelyMatches[b]][mask[b]]
    acc += torch.sum(comparepoints == truepoints)/(3*numReconstructed)
acc /= y.shape[0]
numRec = float(numRec)/y.shape[0]
acc,numRec

(tensor(0.8394), 145.859375)